# 🎬 Director

The `GraphDirector` and `GraphCamera` [behaviors](./Behaviors.ipynb) control and observe
the current contents of the viewport.

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import json
from pathlib import Path

import ipyforcegraph.behaviors as B
import ipyforcegraph.graphs as G
import ipywidgets as W
import traitlets as T

In [ ]:
data = json.loads(Path("./datasets/blocks.json").read_text())

In [ ]:
c = B.GraphCamera(capture_visible=True)
d = B.GraphDirector()
fg = G.ForceGraph(behaviors=[c, d])
fg.source.nodes, fg.source.links = data["nodes"], data["links"]

In [ ]:
xyz = "xyz"
c_kxyz = {
    x: W.FloatSlider(description=x, min=-1000, max=1000, disabled=True) for x in xyz
}
c_kxyz["k"] = W.FloatSlider(description="k", min=0, max=10, step=0.001, disabled=True)
c_capture = W.ToggleButton(description="capture", icon="crop")
c_vis = W.IntText(description="visible", disabled=True)

In [ ]:
d_kxyz = {x: W.FloatSlider(description=x, min=-1000, max=1000) for x in xyz}
d_kxyz["k"] = W.FloatSlider(description="k", min=0, max=5, step=0.001)
durs = ["pan", "zoom"]
d_dur = {dur: W.FloatSlider(description=f"{dur} (s)", max=5) for dur in durs}
btn_action = W.Button(description="action", icon="play", button_style="success")
btn_follow = W.ToggleButton(description="follow", icon="lock", value=True)
sel_user = W.Dropdown(
    description="user",
    options=sorted({d["user"] for d in data["nodes"] if "user" in d}),
)
tmpl_txt = W.Textarea(description="template")
tmpl_nj = B.Nunjucks("")
btn_tmpl = W.ToggleButton(description="use template", icon="filter")
fit_pad = W.FloatSlider(description="padding", min=0, max=200)

In [ ]:
def on_zoomed(*_):
    c_kxyz["k"].value = c.zoom
    if btn_follow.value:
        d_kxyz["k"].value = c.zoom
    for i, v in enumerate(c.center):
        x = xyz[i]
        c_kxyz[x].value = v
        if btn_follow.value:
            d_kxyz[x].value = v


T.dlink((c, "visible"), (c_vis, "value"), len)
T.link((c, "capture_visible"), (c_capture, "value"))
c.observe(on_zoomed, ["zoom", "center"])

In [ ]:
def on_direct(*_):
    with d.hold_sync():
        d.zoom = d_kxyz["k"].value
        d.center = [d_kxyz[x].value for x in xyz]
    d.send_state("zoom")
    d.send_state("center")


[T.link((d, f"{dur}_duration"), (ds, "value")) for dur, ds in d_dur.items()]
btn_action.on_click(on_direct)

In [ ]:
def on_tmpl(*_):
    if not btn_tmpl.value:
        d.fit_nodes = ""
        return
    d.fit_nodes = tmpl_nj


T.dlink(
    (sel_user, "value"), (tmpl_txt, "value"), lambda x: "{{ node.user == '%s' }}" % x
)
T.dlink((d, "fit_padding"), (fit_pad, "value"))
T.dlink((tmpl_txt, "value"), (tmpl_nj, "value"))
btn_tmpl.observe(on_tmpl, "value")

In [ ]:
W.HBox(
    [
        W.VBox(
            [
                W.Label("🎥 Camera"),
                *c_kxyz.values(),
                W.HBox([c_capture, c_vis]),
                W.Label("🎬 Director"),
                W.Tab(
                    [
                        W.VBox(
                            [
                                *d_kxyz.values(),
                                *d_dur.values(),
                                W.HBox([btn_follow, btn_action]),
                            ]
                        ),
                        W.VBox([sel_user, tmpl_txt, fit_pad, btn_tmpl]),
                    ],
                    titles=["Bounds", "Template"],
                ),
            ],
            layout=dict(min_width="25em"),
        ),
        fg,
    ],
    layout=dict(height="100%"),
)

In [ ]:
len(c.visible)